In [ ]:
import re
import torch
import numpy as np
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D,Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler


# **Get Dataset From Drive**
##### Note: Shortcut the folder: [*NN_2023_[NLP Project]*](https://drive.google.com/drive/folders/132HlmBcQUb5nE_jBATbNrObjxe2n19Cy?usp=drive_link) in previous link on your drive before you run this cell

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Specify the file or folder path
# file_path = '/content/drive/MyDrive/Comppetion/train.xlsx'
# test_path="/content/drive/MyDrive/Comppetion/test .csv"

file_path="/content/drive/MyDrive/NN_2023_[NLP Project]/NN'23_arabic_sentiment_analysis_NLP/train.xlsx"
test_path="/content/drive/MyDrive/NN_2023_[NLP Project]/NN'23_arabic_sentiment_analysis_NLP/test.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Load Dataset**

In [ ]:
# Create Data Frame
data = pd.read_excel(file_path)

# **Data Exploration & visualization**

In [ ]:
print(f"Data Shape: {data.shape}")
print(f"\nFirst 6 columns in data:\n {data.head(6)}")

Data Shape: (32036, 2)

First 6 columns in data:
                                   review_description  rating
0  شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...      -1
1  خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...       1
2  تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...      -1
3  فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...       1
4  سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...      -1
5  قعد عشرين سنة يدور على سائق بس اما عن توصيل ال...       0


# **Data Preprocessing**

## Text Preprocessing

### Clean Text

In [ ]:
# Remove duplicated
print(data.review_description.duplicated().sum())
data.drop(data[data.review_description.duplicated() == True].index, axis = 0, inplace = True)

726


### Emojis

In [ ]:
emojis = {
    "🙂":"يبتسم",
    "😂":"يضحك",
    "💔":"قلب حزين",
    "❤️":"حب",
    "❤":"حب",
    "😍":"حب",
    "😭":"يبكي",
    "😢":"حزن",
    "😔":"حزن",
    "♥":"حب",
    "💜":"حب",
    "😅":"يضحك",
    "🙁":"حزين",
    "💕":"حب",
    "💙":"حب",
    "😞":"حزين",
    "😊":"سعادة",
    "👏":"يصفق",
    "👌":"احسنت",
    "😴":"ينام",
    "😀":"يضحك",
    "😌":"حزين",
    "🌹":"وردة",
    "🙈":"حب",
    "😄":"يضحك",
    "😐":"محايد",
    "✌":"منتصر",
    "✨":"نجمه",
    "🤔":"تفكير",
    "😏":"يستهزء",
    "😒":"يستهزء",
    "🙄":"ملل",
    "😕":"عصبية",
    "😃":"يضحك",
    "🌸":"وردة",
    "😓":"حزن",
    "💞":"حب",
    "💗":"حب",
    "😑":"منزعج",
    "💭":"تفكير",
    "😎":"ثقة",
    "💛":"حب",
    "😩":"حزين",
    "💪":"عضلات",
    "👍":"موافق",
    "🙏🏻":"رجاء طلب",
    "😳":"مصدوم",
    "👏🏼":"تصفيق",
    "🎶":"موسيقي",
    "🌚":"صمت",
    "💚":"حب",
    "🙏":"رجاء طلب",
    "💘":"حب",
    "🍃":"سلام",
    "☺":"يضحك",
    "🐸":"ضفدع",
    "😶":"مصدوم",
    "✌️":"مرح",
    "✋🏻":"توقف",
    "😉":"غمزة",
    "🌷":"حب",
    "🙃":"مبتسم",
    "😫":"حزين",
    "😨":"مصدوم",
    "🎼 ":"موسيقي",
    "🍁":"مرح",
    "🍂":"مرح",
    "💟":"حب",
    "😪":"حزن",
    "😆":"يضحك",
    "😣":"استياء",
    "☺️":"حب",
    "😱":"كارثة",
    "😁":"يضحك",
    "😖":"استياء",
    "🏃🏼":"يجري",
    "😡":"غضب",
    "🚶":"يسير",
    "🤕":"مرض",
    "‼️":"تعجب",
    "🕊":"طائر",
    "👌🏻":"احسنت",
    "❣":"حب",
    "🙊":"مصدوم",
    "💃":"سعادة مرح",
    "💃🏼":"سعادة مرح",
    "😜":"مرح",
    "👊":"ضربة",
    "😟":"استياء",
    "💖":"حب",
    "😥":"حزن",
    "🎻":"موسيقي",
    "✒":"يكتب",
    "🚶🏻":"يسير",
    "💎":"الماظ",
    "😷":"وباء مرض",
    "☝":"واحد",
    "🚬":"تدخين",
    "💐" : "ورد",
    "🌞" : "شمس",
    "👆" : "الاول",
    "⚠️" :"تحذير",
    "🤗" : "احتواء",
    "✖️": "غلط",
    "📍"  : "مكان",
    "👸" : "ملكه",
    "👑" : "تاج",
    "✔️" : "صح",
    "💌": "قلب",
    "😲" : "مندهش",
    "💦": "ماء",
    "🚫" : "خطا",
    "👏🏻" : "برافو",
    "🏊" :"يسبح",
    "👍🏻": "تمام",
    "⭕️" :"دائره كبيره",
    "🎷" : "ساكسفون",
    "👋": "تلويح باليد",
    "✌🏼": "علامه النصر",
    "🌝":"مبتسم",
    "➿"  : "عقده مزدوجه",
    "💪🏼" : "قوي",
    "📩":  "تواصل معي",
    "☕️": "قهوه",
    "😧" : "قلق و صدمة",
    "🗨": "رسالة",
    "❗️" :"تعجب",
    "🙆🏻": "اشاره موافقه",
    "👯" :"اخوات",
    "©" :  "رمز",
    "👵🏽" :"سيده عجوزه",
    "🐣": "كتكوت",
    "🙌": "تشجيع",
    "🙇": "شخص ينحني",
    "👐🏽":"ايدي مفتوحه",
    "👌🏽": "بالظبط",
    "⁉️" : "استنكار",
    "⚽️": "كوره",
    "🕶" :"حب",
    "🎈" :"بالون",
    "🎀":    "ورده",
    "💵":  "فلوس",
    "😋":  "جائع",
    "😛":  "يغيظ",
    "😠":  "غاضب",
    "✍🏻":  "يكتب",
    "🌾":  "ارز",
    "👣":  "اثر قدمين",
    "❌":"رفض",
    "🍟":"طعام",
    "👬":"صداقة",
    "🐰":"ارنب",
    "☂":"مطر",
    "⚜":"مملكة فرنسا",
    "🐑":"خروف",
    "🗣":"صوت مرتفع",
    "👌🏼":"احسنت",
    "☘":"مرح",
    "😮":"صدمة",
    "😦":"قلق",
    "⭕":"الحق",
    "✏️":"قلم",
    "ℹ":"معلومات",
    "🙍🏻":"رفض",
    "⚪️":"نضارة نقاء",
    "🐤":"حزن",
    "💫":"مرح",
    "💝":"حب",
    "🍔":"طعام",
    "❤︎":"حب",
    "✈️":"سفر",
    "🏃🏻‍♀️":"يسير",
    "🍳":"ذكر",
    "🎤":"مايك غناء",
    "🎾":"كره",
    "🐔":"دجاجة",
    "🙋":"سؤال",
    "📮":"بحر",
    "💉":"دواء",
    "🙏🏼":"رجاء طلب",
    "💂🏿 ":"حارس",
    "🎬":"سينما",
    "♦️":"مرح",
    "💡":"قكرة",
    "‼":"تعجب",
    "👼":"طفل",
    "🔑":"مفتاح",
    "♥️":"حب",
    "🕋":"كعبة",
    "🐓":"دجاجة",
    "💩":"معترض",
    "👽":"فضائي",
    "☔️":"مطر",
    "🍷":"عصير",
    "🌟":"نجمة",
    "☁️":"سحب",
    "👃":"معترض",
    "🌺":"مرح",
    "🔪":"سكينة",
    "♨":"سخونية",
    "👊🏼":"ضرب",
    "✏":"قلم",
    "🚶🏾‍♀️":"يسير",
    "👊":"ضربة",
    "◾️":"وقف",
    "😚":"حب",
    "🔸":"مرح",
    "👎🏻":"لا يعجبني",
    "👊🏽":"ضربة",
    "😙":"حب",
    "🎥":"تصوير",
    "👉":"جذب انتباه",
    "👏🏽":"يصفق",
    "💪🏻":"عضلات",
    "🏴":"اسود",
    "🔥":"حريق",
    "😬":"عدم الراحة",
    "👊🏿":"يضرب",
    "🌿":"ورقه شجره",
    "✋🏼":"كف ايد",
    "👐":"ايدي مفتوحه",
    "☠️":"وجه مرعب",
    "🎉":"يهنئ",
    "🔕" :"صامت",
    "😿":"وجه حزين",
    "☹️":"وجه يائس",
    "😘" :"حب",
    "😰" :"خوف و حزن",
    "🌼":"ورده",
    "💋":  "بوسه",
    "👇":"لاسفل",
    "❣️":"حب",
    "🎧":"سماعات",
    "📝":"يكتب",
    "😇":"دايخ",
    "😈":"رعب",
    "🏃":"يجري",
    "✌🏻":"علامه النصر",
    "🔫":"يضرب",
    "❗️":"تعجب",
    "👎":"غير موافق",
    "🔐":"قفل",
    "👈":"لليمين",
    "™":"رمز",
    "🚶🏽":"يتمشي",
    "😯":"متفاجأ",
    "✊":"يد مغلقه",
    "😻":"اعجاب",
    "🙉" :"قرد",
    "👧":"طفله صغيره",
    "🔴":"دائره حمراء",
    "💪🏽":"قوه",
    "💤":"ينام",
    "👀":"ينظر",
    "✍🏻":"يكتب",
    "❄️":"تلج",
    "💀":"رعب",
    "😤":"وجه عابس",
    "🖋":"قلم",
    "🎩":"كاب",
    "☕️":"قهوه",
    "😹":"ضحك",
    "💓":"حب",
    "☄️ ":"نار",
    "👻":"رعب",
    "❎":"خطء",
    "🤮":"حزن",
    '🏻':"احمر",
    "😝":"مرح"
}

### Other stuff

In [ ]:
import re

arabic_puncituation = ['،','؟']
def remove_character(word):
  for char in arabic_puncituation:
    if char in word:
        word = word.replace(char, '')
  return word


def remove_character(word):
    for char in arabic_puncituation:
        if char in word:
            word = word.replace(char, '')  # Removes the specified character from the word
    return word

number = ['٠', '١', '٢', '٣', '٤', '٥', '٦', '٧', '٨', '٩']
def contains_any_number(word):
  for char in number:
      if char in word:
          return False
  return True

def is_arabic_word(word):
    arabic_pattern = re.compile(r'^[؀-ۿ ً-ْ]+$')  # Arabic Unicode range
    return bool(arabic_pattern.match(word))

def remove_duplicates(word):
    # Initialize an empty string to store the result
    result = ""

    # Iterate through the characters in the word
    for i in range(len(word)):
        # If the current character is not equal to the next character, add it to the result
        if i == len(word) - 1 or word[i] != word[i + 1]:
            result += word[i]

    return result

def checkemojie(text):
    result_text = ""
    for char in text:
        if char in emojis:
            result_text += " " + emojis[char] + " "
        else:
            result_text += char
    return result_text

def remove_stopwords_and_punctuation(text):
    stop_words = set(stopwords.words('arabic'))

    words = checkemojie(text)
    words = words.split()
    words = [remove_duplicates(word) for word in words]
    filtered_words = [word for word in words if word not in stop_words and word not in string.punctuation and is_arabic_word(word) == True and contains_any_number(word) == True]
    modified_list = [remove_character(word) for word in filtered_words]

    return ' '.join(modified_list)

In [ ]:
import nltk
import string
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('snowball_data')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


print(data.head(10))

data['Token'] = data['review_description'].apply(remove_stopwords_and_punctuation)


print(data.head(10))

print(data.iloc[19])


                                  review_description  rating
0  شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...      -1
1  خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...       1
2  تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...      -1
3  فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...       1
4  سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...      -1
5  قعد عشرين سنة يدور على سائق بس اما عن توصيل ال...       0
6                                         احلئ تطبيق       1
7                                      رائع واو مدهش       1
8  مکو بس البحرین وعمان وغیرهه بس العراق مکو یعنی...      -1
9                    تطبيق جميل يستاهل الخمس نجوم👍👍👍       1


[nltk_data] Downloading package snowball_data to /root/nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                  review_description  rating  \
0  شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...      -1   
1  خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...       1   
2  تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...      -1   
3  فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...       1   
4  سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...      -1   
5  قعد عشرين سنة يدور على سائق بس اما عن توصيل ال...       0   
6                                         احلئ تطبيق       1   
7                                      رائع واو مدهش       1   
8  مکو بس البحرین وعمان وغیرهه بس العراق مکو یعنی...      -1   
9                    تطبيق جميل يستاهل الخمس نجوم👍👍👍       1   

                                               Token  
0  شركه زباله سواقين بتبرشم مفيش حتي رقم لشكاوي ا...  
1  خدمة الدفع طريق الكي نت توقفت عندي اصبح فقط ال...  
2  تطبيق غبي جاري حذفه  عاملين اكواد خصم نستخدمها...  
3  فعلا تطبيق متاز فى امكانية يتيح لمستخدم التطبي...  
4          سيء جدا  

In [ ]:
print(data.iloc[9])

print(data.iloc[9]['Token'])


review_description                   تطبيق جميل يستاهل الخمس نجوم👍👍👍
rating                                                             1
Token                 تطبيق جميل يستاهل الخمس نجوم موافق موافق موافق
Name: 9, dtype: object
تطبيق جميل يستاهل الخمس نجوم موافق موافق موافق


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Create a tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Token'])

# Convert text to sequences of integers
sequences = tokenizer.texts_to_sequences(data['Token'])

# Padding sequences to a maximum length
max_length =  max(map(len, sequences))  # Define the maximum sequence length
print(max_length)

padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

print("Word Index:\n", tokenizer.word_index)
print("\nSequences:\n", sequences)
print("\nPadded Sequences:\n", padded_sequences)


286
Word Index:
 {'حب': 1, 'جدا': 2, 'تطبيق': 3, 'التطبيق': 4, 'موافق': 5, 'متاز': 6, 'الطلب': 7, 'برنامج': 8, 'البرنامج': 9, 'انا': 10, 'مش': 11, 'جميل': 12, 'رائع': 13, 'حلو': 14, 'المطاعم': 15, 'التوصيل': 16, 'طلبات': 17, 'الي': 18, 'اله': 19, 'مره': 20, 'خدمة': 21, 'توصيل': 22, 'يضحك': 23, 'ان': 24, 'فاشل': 25, 'المطعم': 26, 'واله': 27, 'اطلب': 28, 'افضل': 29, 'طلبت': 30, 'جيد': 31, 'او': 32, 'علي': 33, 'شي': 34, 'شكرا': 35, 'اي': 36, 'اكثر': 37, 'انصح': 38, 'سيء': 39, 'روعه': 40, 'غضب': 41, 'يوصل': 42, 'خدمه': 43, 'العملاء': 44, 'حزن': 45, 'مطعم': 46, 'طلب': 47, 'فى': 48, 'يوجد': 49, 'مطاعم': 50, 'يكون': 51, 'انه': 52, 'احسن': 53, 'غاضب': 54, 'يعني': 55, 'تم': 56, 'جداً': 57, 'احسنت': 58, 'عشان': 59, 'مرة': 60, 'سيئ': 61, 'ليش': 62, 'مو': 63, 'ليه': 64, 'سريع': 65, 'متازة': 66, 'الى': 67, 'الوقت': 68, 'سئ': 69, 'كتير': 70, 'خصم': 71, 'وقت': 72, 'يتم': 73, 'الطلبات': 74, 'الاكل': 75, 'اول': 76, 'متازه': 77, 'موقع': 78, 'كثير': 79, 'شركة': 80, 'الموقع': 81, 'زي': 82, 'ارجو': 83, 'وا

In [ ]:
padded_sequences = pd.DataFrame(padded_sequences)


In [ ]:
print(len(tokenizer.word_index))


35213


In [ ]:

for i in tokenizer.word_index:
  print(i)

Streaming output truncated to the last 5000 lines.
مرق
جاره
اطبخو
بيوتكم
يشلكم
والنسوان
النائمات
لشناص
منتلز
انتوم
واصلك
نجو
جرار
أعرفها
وجزا
تنفذوا
بيتقال
حًلَوٌ
جّـدٍآ
أكثرهم
السياقه
التغليف
الكوادر
بيشتغلوا
لطاف
بتلغى
وشات
وطالعة
رحاب
ادايقت
متوقعة
وخذفت
بمعاملة
واندم
ترده
وعناية
بنظافة
مرحباسلام
وبالطلب
مطبقين
شأن
مايقرق
بقمت
يكفيكم
عمالا
ويغطي
اريحيه
اليرموك
عفست
وتسجيل
ڜ
اهتمامته
التراخي
الخارجيه
بتسافرو
دوبني
بينزلي
وادامت
تعملولي
اندجو
وبيرفض
استفادت
انهارده
َلا
بيقلوا
الوزن
واسهال
ومعروفه
الأم
إسألو
مديونين
إسألوني
بعرفها
سراقين
ومتوفره
مسم
منتظرساعه
انامشيت
يخرجك
اجيد
كفائه
مؤدبين
الكويس
وخفت
ابعتوا
وماوصلوه
ولايردون
مايستقبل
فيوم
لسحاتنعتبصؤب
هنتواصل
مطورو
القاعدة
اصلاحه
حيعرف
الحوسة
ماتواصل
ليمثلوا
وبيسيئوا
بأسلوبه
رابعة
مسؤله
لشخصين
اوركب
مركب
تكس
قلتلى
وشخة
متكتك
خالية
وشغالة
بمزاجها
حكومة
تردها
الموعيد
ومتباعدة
الاشخص
قاشل
واو؛
صدف
وترتيب
اخفائها
البرازيل
مودب
السائف
نتلةو
أفتحه
خيانة
لضمير
جدل
اعطاءهم
وعدوا
بعملولي
حضر
بنضلش
نوكل
الدليفيرى
اتابعه
بوجد
رأعا
لاغع
ومطاعمها

In [ ]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(data['rating'])
print(f"Encoded Labels:\n {encoded_labels}")

Encoded Labels:
 [0 2 0 ... 0 0 2]


In [ ]:
max_words = 10000

# Define the LSTM model with additional layers
model = Sequential()

# Add an Embedding layer for word embeddings
embedding_dim = 100  # Experiment with a larger embedding dimension
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=padded_sequences.shape[1]))

# Add an LSTM layer with dropout for sequential processing
lstm_units = 50  # Adjust the number of LSTM units based on your needs
model.add(LSTM(units=lstm_units, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))

# Add a 1D Convolutional layer with dropout and L2 regularization
conv_filters = 100
kernel_size = 3
model.add(Conv1D(filters=conv_filters, kernel_size=kernel_size, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))

# Add a Global MaxPooling layer
model.add(GlobalMaxPooling1D())

# Add a Dense layer with dropout and L2 regularization
dense_units = 50  # Adjust the number of units based on your needs
num_classes = len(np.unique(encoded_labels))
model.add(Dense(units=dense_units, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))

# Add the output layer
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model with an adaptive learning rate
initial_learning_rate = 0.001
optimizer = Adam(learning_rate=initial_learning_rate)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model with early stopping and learning rate scheduling
batch_size = 150  # Experiment with larger batch sizes
epochs = 3

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_schedule = LearningRateScheduler(lambda epoch: initial_learning_rate * 0.9 ** epoch)

history = model.fit(padded_sequences, encoded_labels, batch_size=batch_size, epochs=epochs, callbacks=[early_stopping, lr_schedule])

# # Evaluate the model on the test set
# test_loss, test_accuracy = model.evaluate(X_test, y_test)
# print(f"Test Accuracy: {test_accuracy}")

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 286, 100)          1000000   
                                                                 
 lstm_4 (LSTM)               (None, 286, 50)           30200     
                                                                 
 conv1d_4 (Conv1D)           (None, 284, 100)          15100     
                                                                 
 dropout_8 (Dropout)         (None, 284, 100)          0         
                                                                 
 global_max_pooling1d_4 (Gl  (None, 100)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                      

209/209 [==============================] - 229s 1s/step - loss: 1.0364 - accuracy: 0.7332 - lr: 0.0010
Epoch 2/3
110/209 [==============>...............] - ETA: 1:42 - loss: 0.5326 - accuracy: 0.8473

In [ ]:
test_path="/content/drive/MyDrive/NN_2023_[NLP Project]/NN'23_arabic_sentiment_analysis_NLP/test .csv"
test_data=pd.read_csv(test_path)

x_test=test_data["review_description"]
ids=test_data["ID"]
reviews_test=pd.DataFrame(x_test)
x_test
# reviews_test['review_description'], reviews_test['emojis'] = zip(*reviews_test['review_description'].apply(clean_and_extract_emojis))




In [ ]:
x_test =x_test.apply(remove_stopwords_and_punctuation)
x_test

In [ ]:

# Convert text to sequences of integers
sequences = tokenizer.texts_to_sequences(x_test)


padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

print("Word Index:\n", tokenizer.word_index)
print("\nSequences:\n", sequences)
print("\nPadded Sequences:\n", padded_sequences)

x_test = pd.DataFrame(padded_sequences)



In [ ]:
prediction=model.predict(x_test)
import numpy as np
p=np.argmax(prediction,1)
print(p)

In [ ]:
for i in range(len(p)):
  if p[i]==0:
    p[i]=-1
  if p[i]==1:
    p[i]=0
  if p[i]==2:
    p[i]=1
    print(p)

In [ ]:
submission=pd.DataFrame()
submission['ID']=ids
submission['rating']=p
submission.to_csv('pred1.csv',index=False)